In [57]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic 
from statistics import mean 
import json
from datetime import datetime
#from shapely.geometry import Point
from geojson import Point, Feature, FeatureCollection, dump

In [58]:
missing_curb_ramps = pd.read_csv('missing_curb_ramp_locations.tsv.txt', delimiter='\t')

In [59]:
missing_curb_ramps.head()

,whichEndpoint,segmentId,directionInd,streetName,startLat,startLong,endLat,endLong,whichArcgisFile,roadGrade,geoJson,updateTs,roadObjectId
0,end,6465,S,N Alois St,43.0439,-87.9743,43.0487,-87.9725,right,0.001502,"{""id"": 6465, ""bbox"": [-87.97462342515934, 43.0...",2020-03-15 22:07:23,0
1,start,22063,N,N Alois St,43.0439,-87.9744,43.0488,-87.9726,left,0.001502,"{""id"": 22063, ""bbox"": [-87.97476229221513, 43....",2020-03-15 22:55:44,0
2,end,21095,S,W St Paul Ave,43.0346,-87.9175,43.0347,-87.9268,left,0.008067,"{""id"": 21095, ""bbox"": [-87.92682478384663, 43....",2020-03-15 22:52:33,0
3,start,5725,N,W Tory Hill St,43.0373,-87.9242,43.0370,-87.9259,right,0.009910,"{""id"": 5725, ""bbox"": [-87.92595318699416, 43.0...",2020-03-15 22:04:58,0
4,start,14186,W,N 40th St,43.1290,-87.9624,43.1311,-87.9625,right,0.005562,"{""id"": 14186, ""bbox"": [-87.96250702045818, 43....",2020-03-15 22:31:42,0


In [79]:
def make_missing_marker(row):
    if row['whichEndpoint']=='start':
        mcr_point = (row['startLong'], row['startLat'])
    else:
        mcr_point = (row['endLong'], row['endLat'])
    
    sidewalk_points = json.loads(row.geoJson)['geometry']['coordinates']
    try:
        # NOTE: this doesn't work for a small subset of points who have
        # multiple coordinate lists in their geometry. 
        # I am ignoring those points for now and just letting them return nothing
        dists = {geodesic(point, mcr_point).miles: point for point in sidewalk_points}
        for i in sorted(dists.keys()):
            closest_point = dists[i]
        return closest_point
    except:
        return ''

missing_curb_ramps['mcr_point'] = missing_curb_ramps.apply(make_missing_marker, axis=1)

In [81]:
missing_curb_ramps.head()

,whichEndpoint,segmentId,directionInd,streetName,startLat,startLong,endLat,endLong,whichArcgisFile,roadGrade,geoJson,updateTs,roadObjectId,mcr_point
0,end,6465,S,N Alois St,43.0439,-87.9743,43.0487,-87.9725,right,0.001502,"{""id"": 6465, ""bbox"": [-87.97462342515934, 43.0...",2020-03-15 22:07:23,0,"[-87.97462342515934, 43.046051778445495]"
1,start,22063,N,N Alois St,43.0439,-87.9744,43.0488,-87.9726,left,0.001502,"{""id"": 22063, ""bbox"": [-87.97476229221513, 43....",2020-03-15 22:55:44,0,"[-87.97258861249311, 43.04876603996699]"
2,end,21095,S,W St Paul Ave,43.0346,-87.9175,43.0347,-87.9268,left,0.008067,"{""id"": 21095, ""bbox"": [-87.92682478384663, 43....",2020-03-15 22:52:33,0,"[-87.91755128151343, 43.03459837492203]"
3,start,5725,N,W Tory Hill St,43.0373,-87.9242,43.0370,-87.9259,right,0.009910,"{""id"": 5725, ""bbox"": [-87.92595318699416, 43.0...",2020-03-15 22:04:58,0,"[-87.92595318699416, 43.037026506588006]"
4,start,14186,W,N 40th St,43.1290,-87.9624,43.1311,-87.9625,right,0.005562,"{""id"": 14186, ""bbox"": [-87.96250702045818, 43....",2020-03-15 22:31:42,0,"[-87.96250702045818, 43.131147219913686]"


In [92]:
point_list = []
for point in missing_curb_ramps['mcr_point'].values:
    if type(point) == list:
        feature = Feature(geometry=Point((point[1], point[0])))
        point_list.append(feature)
    
fc = FeatureCollection(point_list)

In [93]:
with open('missing_curb_ramps.geojson', 'w') as outfile:
        dump(fc, outfile)